In [ ]:
# %%
print("OK!")

# %%
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_pinecone import PineconeVectorStore
import pinecone
from pinecone import Pinecone, ServerlessSpec
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
import os
import torch

os.chdir('c:/Users/gt260/End-to-end-Medical-Chatbot-using-Llama2')
print(os.getcwd())

# %%
PINECONE_API_KEY = "122f095f-ea22-497c-99a0-2994b23f687d"
# PINECONE_API_ENV = "quickstart"


# from pinecone import Pinecone, ServerlessSpec

# %%
#Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

# %%
extracted_data = load_pdf("data/")

# %%
# extracted_data

# %%
#Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

# %%
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

# %%
# text_chunks

# %%
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

# %%
embeddings = download_hugging_face_embeddings();

# %%
# embeddings

# %%
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

# %%
query_result

# %%
os.environ['PINECONE_API_KEY'] = PINECONE_API_KEY
index_name = "medical-chatbot"
pc = Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index(index_name)

# %%

# texts = ["Tonight, I call on the Senate to: Pass the Freedom to Vote Act.", "ne of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court.", "One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence."]

# docsearch = PineconeVectorStore.from_texts(texts, embedding = embeddings, index_name=index_name)
# docsearch.add_texts([t.page_content for t in text_chunks])

# %%
#If we already have an index we can load it like this
docsearch = PineconeVectorStore.from_existing_index(index_name, embeddings)
query  = "Why am i feeling sick in a dusty room?"
docs = docsearch.similarity_search(query, k=3)
print("Result", docs)

# %%
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

# %%
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}


device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")
# %%
llm=CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

# %%


# %%
qa = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs
    )

# %%
while True:
    user_input=input(f"Input Prompt:")
    result=qa({"query": user_input})
    print("Response : ", result["result"])

# %%



